# Change the ecoinvent version of the mapping file

In [1]:
import pandas as pd
import bw2data as bd
from mescal import *
import itertools

In [2]:
esm_location = 'CA-QC'
ei_version_from = '3.8'
ei_version_to = '3.9.1'

In [3]:
bd.projects.set_current(f"ecoinvent{ei_version_to}")

In [4]:
name_comp_db = f'ecoinvent_cutoff_{ei_version_from}_remind_SSP2-Base_2020_comp_{esm_location}'

In [5]:
mapping = pd.read_csv(f"energyscope_data/{esm_location}/mapping_{ei_version_from}_linked.csv")

In [6]:
updated_mapping, unit_changes = change_ecoinvent_version_mapping(mapping, ei_version_from, ei_version_to, name_comp_db)

BATTERY Construction
Old: battery, Li-ion, rechargeable, prismatic - market for battery, Li-ion, rechargeable, prismatic - GLO
New: battery, Li-ion, LiMn2O4, rechargeable, prismatic - market for battery, Li-ion, LiMn2O4, rechargeable, prismatic - GLO
CEMENT_PROD Operation
Old: cement, alternative constituents 21-35% - cement production, alternative constituents 21-35% - RoW
New: cement, CEM II/B - cement production, CEM II/B - RoW
CEMENT_PROD_HP Operation
Old: cement, alternative constituents 21-35% - cement production, alternative constituents 21-35% - RoW
New: cement, CEM II/B - cement production, CEM II/B - RoW
ELEC_STO Construction
Old: battery, Li-ion, rechargeable, prismatic - market for battery, Li-ion, rechargeable, prismatic - GLO
New: battery, Li-ion, LiMn2O4, rechargeable, prismatic - market for battery, Li-ion, LiMn2O4, rechargeable, prismatic - GLO
NG_COMP_HE Operation
Old: natural gas, high pressure - market for natural gas, high pressure - CA-QC
New: natural gas, high pr

In [7]:
updated_mapping.head()

,Name,Type,Product,Activity,Location,Database
0,AEC_OG,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
1,AEC_OG_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
2,AEC_OG_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
3,AEC_OG_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
4,AEC_OG_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...


In [8]:
base_db = concatenate_databases(list(updated_mapping.Database.unique()), create_pickle=True)
unlinked = test_mapping_file(updated_mapping, base_db)

Mapping successfully linked to the database


In [9]:
if len(unlinked) == 0:
    updated_mapping.to_csv(f"energyscope_data/{esm_location}/mapping_{ei_version_to}.csv", index=False)

# Adapt the unit conversion file corresponding to the new mapping file

In [10]:
unit_conversion = pd.read_csv(f"energyscope_data/{esm_location}/unit_conversion_{ei_version_from}.csv")

In [11]:
unit_changes

[[('OCGT_LARGE', 'Operation'),
  ('electricity, medium voltage',
   'natural gas, burned in gas turbine, for compressor station',
   'CA-QC'),
  'kWh',
  'MJ'],
 [('OCGT_SMALL', 'Operation'),
  ('electricity, medium voltage',
   'natural gas, burned in gas turbine, for compressor station',
   'CA-QC'),
  'kWh',
  'MJ']]

In [12]:
# Add new unit conversion factors
new_unit_conversion_factors = {
    ('OCGT_LARGE', 'Operation'): 3.6e6 * 2.5,  # 2.5 MWh NG per MWh electricity
    ('OCGT_SMALL', 'Operation'): 3.6e6 * 2.5,  # 2.5 MWh NG per MWh electricity
}

In [13]:
unit_conversion = update_unit_conversion_file(unit_conversion, unit_changes, new_unit_conversion_factors)

In [14]:
unit_conversion.sort_values(by=['Name', 'Type']).to_csv(f"energyscope_data/{esm_location}/unit_conversion_{ei_version_to}.csv", index=False)